In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

In [18]:
bucket_name = 'bankapplication1234567121212121'
session=boto3.session.Session()
my_region=session.region_name
print(my_region)

ap-south-1


In [19]:
s3 = boto3.resource('s3')
try:
	if my_region=='ap-south-1':
		s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
		print("S3 Bucket created successfully")
except Exception as e:
	print("Error: ", e)

Error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [20]:
prefix = 'xgboost-as-a-built-in-algo'
output_path = f's3://{bucket_name}//{prefix}/output'
print(output_path)

s3://bankapplication1234567121212121//xgboost-as-a-built-in-algo/output


In [24]:
s3_client = session.client('s3')
response = s3_client.list_buckets()
# Output the bucket names
# for my_bucket_object in response.objects.all():
for bucket in response['Buckets']:
      print({bucket["Name"]})

{'bankapplication1234567121212121'}
